In [ ]:
%run Silver_Layer_notebook_sw

StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 58, Finished, Available, Finished)

HR data successfully loaded into the Staging Layer: 'hr_data_staging.csv'


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 054c0c97-9d85-4424-bc88-e1dfd28992a7)

HR data successfully moved to the Bronze Layer: 'hr_data_bronze.parquet'


...


Silver Layer Processing Summary:

Total records before processing: 22227
Null values removed (except termdate): 0
Duplicate records removed: 13
Total records after cleaning: 22214



SynapseWidget(Synapse.DataFrame, 38022ae4-e033-4cb5-be3e-1ef8518e43a8)

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)
 |-- int_id: integer (nullable = true)



SynapseWidget(Synapse.DataFrame, 45485227-e1c4-44d3-b8f4-e2f6e9cb11de)

SynapseWidget(Synapse.DataFrame, 8378d2ce-fc46-45de-9d06-0179a3f584e8)

StatementMeta(, , -1, Finished, , Finished)

MagicUsageError: Fetch content failed for "Files", please make sure the notebook exists. It might be caused by using a workspace folder path, please use Notebook name directly. Notebook path: Files

In [ ]:
spark.sql("SHOW TABLES").show() 


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 59, Finished, Available, Finished)

+------------------+--------------+-----------+
|         namespace|     tableName|isTemporary|
+------------------+--------------+-----------+
|silver_lakehouse_1|hr_data_silver|      false|
+------------------+--------------+-----------+



In [ ]:
# Gold Layer

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date

# Initialize Spark session
spark = SparkSession.builder.appName("HR Data Gold Layer").getOrCreate()


# Paths to Silver and Gold Lakehouse
silver_delta_tblpath= "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lakehouse_1.Lakehouse/Tables/hr_data_silver"
gold_lh_tpath = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/gold_lakehouse_sw1.Lakehouse/Tables/"

# Load Silver Layer Data
df_silver_delta = spark.read.format("delta").load(silver_delta_tblpath)


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 10, Finished, Available, Finished)

In [ ]:
df_silver_delta.columns

StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 60, Finished, Available, Finished)

['id',
 'first_name',
 'last_name',
 'birthdate',
 'gender',
 'race',
 'department',
 'jobtitle',
 'location',
 'hire_date',
 'termdate',
 'location_city',
 'location_state',
 'employee_id']

In [ ]:
# Dimension Table :----> Employee Details

from pyspark.sql.functions import col, concat_ws

DimEmployee = df_silver_delta.select(
    col("id").alias("employee_id"),  # Ensure ID is included
    col("first_name"),
    col("last_name"),
    col("birthdate"),
    col("gender"),
    col("race"),
    col("hire_date"),
    col("termdate").alias("termination_date"),  # Fixed alias typo
    concat_ws(" ", col("first_name"), col("last_name")).alias("full_name")  # Correct function usage
)

df = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/silver_lakehouse_1.Lakehouse/Tables/hr_data_silver")
# display(df)

# Write to Gold Lakehouse as Delta Table

DimEmployee.write.format("delta").mode("overwrite").save(gold_lh_tpath +"//"+"DimEmployee")

print("`DimEmployee` Table Created Successfully in Gold Lakehouse")

# Show the transformed data
# DimEmployee.show(5)
# display(DimEmployee)

StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 75, Finished, Available, Finished)

`DimEmployee` Table Created Successfully in Gold Lakehouse


In [ ]:
from pyspark.sql.functions import col, concat_ws, row_number
from pyspark.sql.window import Window

# Dim Location Table

# Step 1: Extract distinct city/state pairs
distinct_locations = hr_data_silver.select("location_city", "location_state").distinct()

# Step 2: Define window for row_number
window_spec = Window.orderBy("location_city", "location_state")

# Step 3: Generate DimLocation table
DimLocation  = distinct_locations.withColumn(
    "location_id", row_number().over(window_spec) + 100  # Start IDs from 101
).select(
    col("location_id"),
    col("location_city"),
    col("location_state"),
    concat_ws(", ", col("location_city"), col("location_state")).alias("location")
)

# Step 4: Save as Delta Table to Gold Lakehouse
dim_location_path = gold_lh_tpath + "/DimLocation"  # fixed typo in variable name if needed

DimLocation .write.format("delta").mode("overwrite").save(dim_location_path)

print("DimLocation table created and saved successfully")

display(dim_location)


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 73, Finished, Available, Finished)

✅ DimLocation table created and saved successfully!


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d9c7977e-3b16-48bf-873c-3174cfca53ae)

In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Step 1: Extract unique departments
distinct_departments = hr_data_silver.select("department").distinct().na.drop()

# Step 2: Assign department_id starting from 201
window_spec_dept = Window.orderBy("department")

dim_department = distinct_departments.withColumn(
    "department_id", row_number().over(window_spec_dept) + 200
).select(
    col("department_id"),
    col("department")
)

# Step 3: Save as Delta Table to Gold Lakehouse
dim_department_path = gold_lh_fpath + "/DimDepartment"

dim_department.write.format("delta").mode("overwrite").save(dim_department_path)

print("DimDepartment table created and saved successfully!")


StatementMeta(, 957fb5bf-855f-4564-b9dd-dd49330bf169, 123, Finished, Available, Finished)

abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/gold_lakehouse_sw1.Lakehouse/Tables


In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Step 1: Extract unique departments
distinct_departments = hr_data_silver.select("department").distinct().na.drop()

# Step 2: Assign department_id starting from 201
window_spec_dept = Window.orderBy("department")

dim_department = distinct_departments.withColumn(
    "department_id", row_number().over(window_spec_dept) + 0
).select(
    col("department_id"),
    col("department")
)

# Step 3: Save as Delta Table to Gold Lakehouse
dim_department_path = gold_lh_fpath + "/DimDepartment"

dim_department.write.format("delta").mode("overwrite").save(dim_department_path)


StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 80, Finished, Available, Finished)

In [ ]:
# display(dim_department)

StatementMeta(, f99c012d-e64a-45dc-8b41-118760e4271f, 81, Submitted, Running, Running)